# Next Token Prediction – live & spielerisch (Toy-Notebook)

**Didaktisches Ziel:** *LLMs wählen das wahrscheinlichste nächste Token (bzw. sampeln daraus) – nicht „die Wahrheit“.*

In diesem Notebook bauen wir eine **Mini-Version** der letzten Schritte eines LLMs nach:

1. **Logits** (unskalierte Scores) für ein kleines Vokabular  
2. **Softmax** → Wahrscheinlichkeiten  
3. **Temperatur** (Temperature) steuert „Zufälligkeit“  
4. **Argmax** vs. **Sampling** (Ziehen nach Wahrscheinlichkeit)  
5. Mehrere Durchläufe → unterschiedliche Outputs

> **Wichtig:** Das ist kein echtes LLM – aber es zeigt die **entscheidende Mechanik** am Ende der Pipeline.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Für reproduzierbare Zufallszahlen (Sampling)
rng = np.random.default_rng(42)

np.set_printoptions(precision=4, suppress=True)

## 1) Mini-Vokabular & Beispiel-Logits

Wir nehmen ein **Mini-Vokabular** (5–10 Tokens) und tun so, als hätte das Modell für den nächsten Token bereits Logits berechnet.

- **Logits** sind einfach reelle Zahlen (Scores).  
- Höherer Logit → tendenziell höhere Wahrscheinlichkeit nach Softmax.

> In echten Modellen kommen diese Logits aus einer Projektion auf den Vokabularraum (z. B. `z = W_out · h + b`).


In [ ]:
# Mini-Vokabular (Tokens)
vocab = ["Paris", "Berlin", "Rom", "Frankreich", ".", "<EOS>"]

# Beispiel: Logits für "nächstes Token"
# (Diese Zahlen sind frei gewählt – wir wollen nur die Mechanik sehen.)
logits = np.array([2.4, 1.2, 0.3, 2.0, 0.1, -0.5], dtype=float)

print("Vokabular:", vocab)
print("Logits:   ", logits)

## 2) Softmax (numerisch stabil)

Softmax macht aus Logits Wahrscheinlichkeiten:

\[
p_i = \frac{e^{z_i}}{\sum_j e^{z_j}}
\]

**Numerische Stabilität:** Wir subtrahieren das Maximum, damit `exp()` nicht überläuft.


In [ ]:
def softmax(z: np.ndarray) -> np.ndarray:
    """Numerisch stabile Softmax."""
    z = np.asarray(z, dtype=float)
    z_shift = z - np.max(z)          # Stabilitätstrick
    exp_z = np.exp(z_shift)
    return exp_z / np.sum(exp_z)

probs = softmax(logits)
print("Wahrscheinlichkeiten:", probs)
print("Summe:", probs.sum())

### Visualisierung der Wahrscheinlichkeiten

In [ ]:
def plot_probs(tokens, p, title="Wahrscheinlichkeiten (Softmax)"):
    x = np.arange(len(tokens))
    plt.figure(figsize=(8, 3))
    plt.bar(x, p)
    plt.xticks(x, tokens, rotation=20, ha="right")
    plt.ylim(0, max(0.01, float(np.max(p)) * 1.15))
    plt.title(title)
    plt.ylabel("p")
    plt.tight_layout()
    plt.show()

plot_probs(vocab, probs, title="Softmax auf Logits (T = 1.0)")

## 3) Temperatur (Temperature)

Temperatur skaliert Logits **vor** der Softmax:

\[
p_i(T) = \mathrm{softmax}\left(\frac{z_i}{T}\right)
\]

- **T < 1** → Verteilung wird *spitzer* (mehr „deterministisch“)  
- **T = 1** → unverändert  
- **T > 1** → Verteilung wird *flacher* (mehr Variation)

> Viele Systeme nutzen zusätzlich Top-k / Top-p (Nucleus Sampling). Hier bleiben wir bewusst beim Kern.


In [ ]:
def softmax_with_temperature(z: np.ndarray, T: float) -> np.ndarray:
    if T <= 0:
        raise ValueError("Temperatur T muss > 0 sein.")
    return softmax(z / T)

for T in [0.5, 1.0, 1.5, 2.0]:
    pT = softmax_with_temperature(logits, T)
    entropy = float(-(pT * np.log(pT + 1e-12)).sum())
    print(f"T={T:>3}: p_max={pT.max():.4f}, Entropie={entropy:.4f}")

In [ ]:
for T in [0.5, 1.0, 2.0]:
    plot_probs(vocab, softmax_with_temperature(logits, T), title=f"Softmax mit Temperatur T={T}")

## 4) Argmax vs. Sampling

**Argmax** wählt immer den Token mit maximaler Wahrscheinlichkeit:
\[ \hat{i} = \arg\max_i p_i \]

**Sampling** zieht zufällig gemäß der Wahrscheinlichkeitsverteilung:
- Bei gleicher Verteilung kann in verschiedenen Durchläufen **Verschiedenes** herauskommen.


In [ ]:
def argmax_choice(tokens, p):
    idx = int(np.argmax(p))
    return tokens[idx], idx

def sample_choice(tokens, p, rng: np.random.Generator):
    idx = int(rng.choice(len(tokens), p=p))
    return tokens[idx], idx

p = softmax_with_temperature(logits, T=1.0)

am_token, am_idx = argmax_choice(vocab, p)
print("Argmax:", am_token, "(idx=", am_idx, ")")

samples = [sample_choice(vocab, p, rng)[0] for _ in range(15)]
print("Sampling (15x):", samples)

## 5) Mehrere Durchläufe: Wie oft kommt welcher Token?

Wir simulieren viele Samples und zählen Häufigkeiten.  
Damit sieht man: **Seltene Tokens kommen seltener, aber nicht nie.**


In [ ]:
def sample_many(tokens, p, n=2000, rng=None):
    rng = rng or np.random.default_rng()
    idxs = rng.choice(len(tokens), size=n, p=p)
    counts = np.bincount(idxs, minlength=len(tokens))
    rel = counts / n
    return counts, rel

T = 1.2
pT = softmax_with_temperature(logits, T)
counts, rel = sample_many(vocab, pT, n=3000, rng=rng)

print("Temperatur:", T)
print("Theorie p:", pT)
print("Empirie :", rel)

plot_probs(vocab, rel, title=f"Empirische Häufigkeiten (Sampling, n=3000, T={T})")

## 6) Mini-Experiment: „Wahr“ vs. „Wahrscheinlich“

Stell dir vor, du willst, dass nach *„Die Hauptstadt von Frankreich ist“* das Token **„Paris“** kommt.
Wenn die Logits aber so aussehen, dass **„Frankreich“** oder **„.“** auch relativ wahrscheinlich sind,
kann Sampling (je nach Temperatur) gelegentlich „falsch“ wirkende Tokens erzeugen.

**Kernpunkt:** Ein LLM ist kein Wahrheitsautomat, sondern eine Wahrscheinlichkeitsmaschine im Kontext.


In [ ]:
logits2 = logits.copy()
logits2[vocab.index("Paris")] = 1.6
logits2[vocab.index("Frankreich")] = 2.2
logits2[vocab.index(".")] = 1.4

for T in [0.7, 1.0, 1.3]:
    p2 = softmax_with_temperature(logits2, T)
    token_am, _ = argmax_choice(vocab, p2)
    samples2 = [sample_choice(vocab, p2, rng)[0] for _ in range(12)]
    print(f"\nT={T}")
    print("Argmax:", token_am)
    print("Sampling:", samples2)
    plot_probs(vocab, p2, title=f"Variante logits2 – Softmax mit T={T}")

## 7) Aufgaben (für Studierende)

1. **Temperatur-Experiment:** Probiere `T=0.2`, `T=3.0`. Was passiert?  
2. **Logits selbst wählen:** Setze `logits` so, dass zwei Tokens fast gleich wahrscheinlich sind.  
3. **Sampling-Frequenz:** Erhöhe `n` auf 10000. Wie nah ist die Empirie an der Theorie?  
4. (Optional) Implementiere **Top-k Sampling**: setze alle Wahrscheinlichkeiten außer den k größten auf 0 und normiere neu.

---

### Take-away (Merksatz)
**Ein LLM wählt Tokens nach Wahrscheinlichkeiten – nicht nach „Wahrheit“.**
